In [ ]:
# !pip3 install beautifulsoup4
!pip3 install boto3

In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [7]:
response = requests.get('https://1000mostcommonwords.com/1000-most-common-german-words')
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
tr_elements = soup.find_all('tr')
tr_elements

[<tr>
 <td width="134"><strong>Number</strong></td>
 <td width="134"><strong>German</strong></td>
 <td width="134"><strong>in English</strong></td>
 </tr>,
 <tr>
 <td>1</td>
 <td>wie</td>
 <td>as</td>
 </tr>,
 <tr>
 <td>2</td>
 <td>ich</td>
 <td>I</td>
 </tr>,
 <tr>
 <td>3</td>
 <td>seine</td>
 <td>his</td>
 </tr>,
 <tr>
 <td>4</td>
 <td>dass</td>
 <td>that</td>
 </tr>,
 <tr>
 <td>5</td>
 <td>er</td>
 <td>he</td>
 </tr>,
 <tr>
 <td>6</td>
 <td>war</td>
 <td>was</td>
 </tr>,
 <tr>
 <td>7</td>
 <td>für</td>
 <td>for</td>
 </tr>,
 <tr>
 <td>8</td>
 <td>auf</td>
 <td>on</td>
 </tr>,
 <tr>
 <td>9</td>
 <td>sind</td>
 <td>are</td>
 </tr>,
 <tr>
 <td>10</td>
 <td>mit</td>
 <td>with</td>
 </tr>,
 <tr>
 <td>11</td>
 <td>sie</td>
 <td>they</td>
 </tr>,
 <tr>
 <td>12</td>
 <td>sein</td>
 <td>be</td>
 </tr>,
 <tr>
 <td>13</td>
 <td>bei</td>
 <td>at</td>
 </tr>,
 <tr>
 <td>14</td>
 <td>ein</td>
 <td>one</td>
 </tr>,
 <tr>
 <td>15</td>
 <td>haben</td>
 <td>have</td>
 </tr>,
 <tr>
 <td>16</td>
 <td>d

In [16]:
rows = []
for e in tr_elements:
    rows.append([e.find_all('td')[0].text, e.find_all('td')[1].text, e.find_all('td')[2].text])

df = pd.DataFrame(rows, columns=['rank', 'word', 'translation'])

df.to_csv('german_words.csv', index=False)

In [19]:
from translation_config.gender import get_gender_of_word


In [30]:
with open('translation_config/german_words.csv', encoding='utf8') as file:
    df = pd.read_csv(file)

In [29]:
def gender_wrapper(word):
    if not word[0].isupper():
        return None

    obj = get_gender_of_word(word)
    return obj['definite_article']


In [31]:
df['gender'] = df['word'].apply(gender_wrapper)

In [34]:
df.set_index('rank')

,word,translation,gender
rank,,,
1,wie,as,None
2,ich,I,None
3,seine,his,None
4,dass,that,None
5,er,he,None
...,...,...,...
996,Nase,nose,die
997,mehreren,plural,None
998,Zorn,anger,der


In [35]:
df.to_csv('german_words_with_genders.csv', index=False)

## Upload to DynamoDB

In [37]:
import boto3

In [48]:
session = boto3.Session(profile_name='ja')
dynamodb = session.resource('dynamodb', region_name='ap-southeast-2')
table = dynamodb.Table('german_words')

In [61]:
items = df.to_dict('records')
print(df.shape)
df = df.drop_duplicates()
print(df.shape)


# Split items into chunks for batch write
chunk_size = 25  # Max items per batch write
condition_expression = 'attribute_not_exists(rank)'
for i in range(0, len(items)):
    table.put_item(Item=items[i], ConditionExpression=condition_expression)

(1000, 4)
(1000, 4)


TypeError: BatchWriter.put_item() got an unexpected keyword argument 'ConditionExpression'

In [51]:
items = []

while True:
    response = table.scan()
    items.extend(response['Items'])

    if 'LastEvaluatedKey' not in response:
        break

    last_key = response['LastEvaluatedKey']
    response = table.scan(ExclusiveStartKey=last_key)

for item in items:
    print(item)



{'translation': 'book', 'rank': Decimal('251'), 'gender': 'das', 'word': 'Buch'}
{'translation': 'my', 'rank': Decimal('187'), 'gender': None, 'word': 'meine'}
{'translation': 'up', 'rank': Decimal('154'), 'gender': None, 'word': 'nach oben'}
{'translation': 'for', 'rank': Decimal('7'), 'gender': None, 'word': 'für'}
{'translation': 'round', 'rank': Decimal('115'), 'gender': 'die', 'word': 'Runde'}
{'translation': 'sudden', 'rank': Decimal('512'), 'gender': None, 'word': 'plötzlich'}
{'translation': 'group', 'rank': Decimal('286'), 'gender': 'die', 'word': 'Gruppe'}
{'translation': 'decide', 'rank': Decimal('361'), 'gender': None, 'word': 'entscheiden'}
{'translation': 'thousand', 'rank': Decimal('380'), 'gender': None, 'word': 'tausend'}
{'translation': 'cloud', 'rank': Decimal('678'), 'gender': 'die', 'word': 'Wolke'}
{'translation': 'general', 'rank': Decimal('526'), 'gender': None, 'word': 'allgemein'}
{'translation': 'year', 'rank': Decimal('117'), 'gender': 'das', 'word': 'Jahr'}

In [57]:

ranks = sorted([item['rank'] for item in items])

In [58]:
print(ranks)

[Decimal('1'), Decimal('2'), Decimal('3'), Decimal('4'), Decimal('5'), Decimal('6'), Decimal('7'), Decimal('8'), Decimal('9'), Decimal('10'), Decimal('11'), Decimal('12'), Decimal('13'), Decimal('14'), Decimal('15'), Decimal('16'), Decimal('17'), Decimal('18'), Decimal('19'), Decimal('20'), Decimal('21'), Decimal('22'), Decimal('23'), Decimal('24'), Decimal('25'), Decimal('26'), Decimal('27'), Decimal('28'), Decimal('29'), Decimal('30'), Decimal('31'), Decimal('32'), Decimal('33'), Decimal('34'), Decimal('35'), Decimal('36'), Decimal('37'), Decimal('38'), Decimal('39'), Decimal('40'), Decimal('41'), Decimal('42'), Decimal('43'), Decimal('44'), Decimal('45'), Decimal('46'), Decimal('47'), Decimal('48'), Decimal('49'), Decimal('50'), Decimal('51'), Decimal('52'), Decimal('53'), Decimal('54'), Decimal('55'), Decimal('56'), Decimal('57'), Decimal('58'), Decimal('59'), Decimal('60'), Decimal('61'), Decimal('62'), Decimal('63'), Decimal('64'), Decimal('65'), Decimal('66'), Decimal('67'), Dec